In [166]:
import pandas as pd

In [167]:
df_fw = pd.read_csv('../../data/landing/player_stats_12-22_fw.csv', index_col=0)

In [168]:
df_fw

,Date,Season,Round,Venue,Player,Team,Opposition,Status,Match_id,CP,...,FA,AF,SC,CCL,SCL,SI,MG,TO,ITC,T5
1,2012-03-24,2012,Round 1,ANZ Stadium,Adam Kennedy,GWS,Sydney,Home,5343,8,...,0,116,103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-03-24,2012,Round 1,ANZ Stadium,Toby Greene,GWS,Sydney,Home,5343,0,...,0,99,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012-03-24,2012,Round 1,ANZ Stadium,Tomas Bugg,GWS,Sydney,Home,5343,7,...,1,97,87.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012-03-24,2012,Round 1,ANZ Stadium,Rhys Palmer,GWS,Sydney,Home,5343,10,...,3,98,126.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2012-03-24,2012,Round 1,ANZ Stadium,Chad Cornes,GWS,Sydney,Home,5343,8,...,1,111,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98603,2022-09-24,2022,Grand Final,MCG,Justin McInerney,Sydney,Geelong,Away,10750,2,...,0,32,29.0,0.0,0.0,1.0,97.0,1.0,2.0,0.0
98604,2022-09-24,2022,Grand Final,MCG,Lance Franklin,Sydney,Geelong,Away,10750,1,...,1,22,11.0,0.0,0.0,2.0,68.0,1.0,0.0,0.0
98605,2022-09-24,2022,Grand Final,MCG,Sam Reid,Sydney,Geelong,Away,10750,1,...,0,11,17.0,0.0,0.0,1.0,-2.0,0.0,1.0,0.0
98606,2022-09-24,2022,Grand Final,MCG,Hayden McLean,Sydney,Geelong,Away,10750,3,...,0,35,44.0,1.0,0.0,3.0,32.0,0.0,1.0,0.0


In [169]:
df_fw.columns

Index(['Date', 'Season', 'Round', 'Venue', 'Player', 'Team', 'Opposition',
       'Status', 'Match_id', 'CP', 'UP', 'ED', 'DE', 'CM', 'GA', 'MI5',
       'One.Percenters', 'BO', 'TOG', 'K', 'HB', 'D', 'M', 'G', 'B', 'T', 'HO',
       'I50', 'CL', 'CG', 'R50', 'FF', 'FA', 'AF', 'SC', 'CCL', 'SCL', 'SI',
       'MG', 'TO', 'ITC', 'T5'],
      dtype='object')

In [170]:
df_fw = df_fw[['Season', 'Round', 'Venue', 'Player', 'Team', 'Opposition',
       'Status', 'Match_id', 'SC'
       ]]

In [171]:
df = pd.read_parquet('../../data/raw/cleaned_stats_12-22_fixed_bv.parquet')

In [172]:
df = df[['match_id', 'match_home_team', 'match_round', 'season', 'player_team']].drop_duplicates()

In [173]:
print(set(df_fw.Team.unique()) - set(df.player_team.unique()))
print(set(df.player_team.unique()) - set(df_fw.Team.unique()))

{'GWS', 'Brisbane'}
{'Greater Western Sydney', 'Brisbane Lions'}


In [180]:
df_fw.replace({
    'Team': {
        'Brisbane': 'Brisbane Lions',
        'GWS': 'Greater Western Sydney'
    },
    'Opposition': {
        'Brisbane': 'Brisbane Lions',
        'GWS': 'Greater Western Sydney'
    }
}, inplace=True)

/var/folders/_d/tnkg6yhj7xj4rxgh61bmdwtr0000gn/T/ipykernel_97804/4017094053.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fw.replace({


In [181]:
def get_home_team(row):

    if row['Status'] == 'Home':
        return row['Team']
    else:
        return row['Opposition']

In [182]:
df_fw['match_home_team'] = df_fw.apply(get_home_team, axis=1)

/var/folders/_d/tnkg6yhj7xj4rxgh61bmdwtr0000gn/T/ipykernel_97804/168857484.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fw['match_home_team'] = df_fw.apply(get_home_team, axis=1)


In [184]:
df_fw.Round.unique()

array(['Round 1', 'Round 2', 'Round 3', 'Round 4', 'Round 5', 'Round 6',
       'Round 7', 'Round 8', 'Round 9', 'Round 10', 'Round 11',
       'Round 12', 'Round 13', 'Round 14', 'Round 15', 'Round 16',
       'Round 17', 'Round 18', 'Round 19', 'Round 20', 'Round 21',
       'Round 22', 'Round 23', 'Qualifying Final', 'Elimination Final',
       'Semi Final', 'Preliminary Final', 'Grand Final',
       'Preliminary Finals'], dtype=object)

In [185]:
df_fw['number'] = df_fw['Round'].str.extract('(\d+)')

/var/folders/_d/tnkg6yhj7xj4rxgh61bmdwtr0000gn/T/ipykernel_97804/3418338132.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fw['number'] = df_fw['Round'].str.extract('(\d+)')


In [186]:
df_fw.isna().sum()

Season                0
Round                 0
Venue                 0
Player                0
Team                  0
Opposition            0
Status                0
Match_id              0
SC                  134
match_home_team       0
number             4380
dtype: int64

In [187]:
df_fw.loc[df_fw.number.isna()].Round.unique()

array(['Qualifying Final', 'Elimination Final', 'Semi Final',
       'Preliminary Final', 'Grand Final', 'Preliminary Finals'],
      dtype=object)

In [188]:
# remove finals matches
df_fw = df_fw.loc[~df_fw.number.isna()]

In [189]:
df_fw = df_fw.astype({'number':'int'})

In [190]:
# 3 matches with missing SC values
df_fw.loc[df_fw.isna().any(axis=1)].drop_duplicates('Match_id')

,Season,Round,Venue,Player,Team,Opposition,Status,Match_id,SC,match_home_team,number
83192,2021,Round 9,Marvel Stadium,Brad Crouch,St Kilda,Geelong,Home,10404,NaN,St Kilda,9
84898,2021,Round 13,Optus Stadium,Jack Redden,West Coast,Richmond,Home,10443,NaN,West Coast,13
89269,2022,Round 1,MCG,Christian Petracca,Melbourne,Western Bulldogs,Home,10544,NaN,Melbourne,1


# 2021 Round 9 St Kilda v Geelong

In [191]:
url = 'https://www.footywire.com/afl/footy/supercoach_round?year=2021&round=9&p=&s=T'

In [192]:
download = pd.read_html(url)

In [193]:
# there are 7 columns in target dataframe:
# rank, player, team, current salary, round salary, round score, round value

for i in range(len(download)):

    if len(download[i].columns) == 7:
        print(i)


10


In [194]:
r9_2021 = download[10]

In [195]:
r9_2021

,0,1,2,3,4,5,6
0,Rank,Player,Team,Current Salary,2021 R9 Salary,2021 R9 Score,*2021 R9 Value
1,1,Jy Simpkin,Kangaroos,"$415,400","$432,600",164,37.9
2,2,Todd Goldstein,Kangaroos,"$398,500","$467,300",150,32.1
3,3,Sean Darcy,Dockers,"$553,800","$509,700",146,28.6
4,4,Dayne Zorko,Lions,"$448,100","$492,000",140,28.5
...,...,...,...,...,...,...,...
411,404,Jake Melksham,Demons,"$270,600","$280,000",0,0.0
412,404,Zac Langdon,Eagles,"$264,300","$201,100",0,0.0
413,404,Jack Madgen,Magpies,"$289,100","$298,400",0,0.0
414,404,James Bell,Swans,"$178,900","$267,300",0,0.0


In [196]:
convert_team = {
    'Blues': 'Carlton',
    'Bombers': 'Essendon',
    'Bulldogs': 'Western Bulldogs',
    'Cats': 'Geelong',
    'Crows': 'Adelaide',
    'Demons': 'Melbourne',
    'Dockers': 'Fremantle',
    'Eagles': 'West Coast',
    'Giants': 'Greater Western Sydney',
    'Hawks': 'Hawthorn',
    'Kangaroos': 'North Melbourne',
    'Lions': 'Brisbane Lions',
    'Magpies': 'Collingwood',
    'Power': 'Port Adelaide',
    'Saints': 'St Kilda',
    'Suns': 'Gold Coast',
    'Swans': 'Sydney',
    'Tigers': 'Richmond'
}

def clean_df(df, team_1, team_2):

    df = df.drop(columns=[0, 3, 4, 6])
    
    df.columns = ['player', 'player_team', 'SC']

    df = df.replace({'player_team': convert_team})

    
    return df.query('(player_team == @team_1) | (player_team == @team_2)').sort_values('player')

In [197]:
r9_2021 = clean_df(r9_2021, 'St Kilda', 'Geelong')

In [198]:
len(r9_2021)

46

In [199]:
idx_to_change = df_fw.loc[((df_fw['number'] == 9) & 
                          (df_fw['Season'] == 2021) &
                          ((df_fw['Team'] == "Geelong") |
                          (df_fw['Team'] == "St Kilda")))].sort_values('Player').index

# player missing
len(idx_to_change)

45

In [200]:
# Max Holmes was not subbed into the match therefore he is removed
r9_2021.drop(408, inplace=True)

In [201]:
df_fw.loc[idx_to_change, 'SC'] = r9_2021['SC'].tolist()

In [202]:
# looks good
df_fw.loc[idx_to_change]

,Season,Round,Venue,Player,Team,Opposition,Status,Match_id,SC,match_home_team,number
83214,2021,Round 9,Marvel Stadium,Ben Long,St Kilda,Geelong,Home,10404,8,St Kilda,9
83192,2021,Round 9,Marvel Stadium,Brad Crouch,St Kilda,Geelong,Home,10404,116,St Kilda,9
83233,2021,Round 9,Marvel Stadium,Bradley Close,Geelong,St Kilda,Away,10404,78,St Kilda,9
83196,2021,Round 9,Marvel Stadium,Bradley Hill,St Kilda,Geelong,Home,10404,88,St Kilda,9
83224,2021,Round 9,Marvel Stadium,Brandan Parfitt,Geelong,St Kilda,Away,10404,71,St Kilda,9
83210,2021,Round 9,Marvel Stadium,Callum Wilkie,St Kilda,Geelong,Home,10404,37,St Kilda,9
83215,2021,Round 9,Marvel Stadium,Cameron Guthrie,Geelong,St Kilda,Away,10404,136,St Kilda,9
83211,2021,Round 9,Marvel Stadium,Daniel Butler,St Kilda,Geelong,Home,10404,78,St Kilda,9
83207,2021,Round 9,Marvel Stadium,Dougal Howard,St Kilda,Geelong,Home,10404,56,St Kilda,9
83228,2021,Round 9,Marvel Stadium,Esava Ratugolea,Geelong,St Kilda,Away,10404,84,St Kilda,9


# Round 13 2021 West Coast v Richmond

In [203]:
url = 'https://www.footywire.com/afl/footy/supercoach_round?year=2021&round=13&p=&s=T'

download = pd.read_html(url)

In [204]:
r13_2021 = download[10]

In [205]:
r13_2021

,0,1,2,3,4,5,6
0,Rank,Player,Team,Current Salary,2021 R13 Salary,2021 R13 Score,*2021 R13 Value
1,1,Scott Pendlebury,Magpies,"$498,500","$382,900",167,43.6
2,2,Dougal Howard,Saints,"$375,100","$378,400",160,42.3
3,3,Jeremy Cameron,Cats,"$363,600","$398,600",151,37.9
4,4,Joshua Kelly,Giants,"$469,900","$562,300",147,26.1
...,...,...,...,...,...,...,...
319,314,Curtis Taylor,Kangaroos,"$298,900","$222,900",0,0.0
320,314,Connor Blakely,Dockers,"$123,900","$219,200",0,0.0
321,314,Hugo Ralphsmith,Tigers,"$136,700","$134,300",0,0.0
322,314,Xavier O'Halloran,Giants,"$317,500","$261,000",0,0.0


In [206]:
r13_2021 = clean_df(r13_2021, 'Richmond', 'West Coast')

In [207]:
len(r13_2021)

46

In [208]:
idx_to_change = df_fw.loc[((df_fw['number'] == 13) & 
                          (df_fw['Season'] == 2021) &
                          ((df_fw['Team'] == "Richmond") |
                          (df_fw['Team'] == "West Coast")))].sort_values('Player').index

# player missing
len(idx_to_change)

44

In [209]:
# need to convert Coleman-Jones to C-Jones
# drop Ralpsmith and O'Neill
set(r13_2021.player) - set(df_fw.loc[idx_to_change, 'Player'])

{'Callum Coleman-Jones', 'Hugo Ralphsmith', "Xavier O'Neill"}

In [210]:
r13_2021.drop([321, 315], inplace=True)
r13_2021.loc[119, 'player'] = 'Callum C-Jones'

In [211]:
df_fw.loc[idx_to_change, 'SC'] = r13_2021['SC'].tolist()

In [212]:
# looks good
df_fw.loc[idx_to_change]

,Season,Round,Venue,Player,Team,Opposition,Status,Match_id,SC,match_home_team,number
84902,2021,Round 13,Optus Stadium,Andrew Gaff,West Coast,Richmond,Home,10443,80,West Coast,13
84921,2021,Round 13,Optus Stadium,Bachar Houli,Richmond,West Coast,Away,10443,99,West Coast,13
84937,2021,Round 13,Optus Stadium,Callum C-Jones,Richmond,West Coast,Away,10443,81,West Coast,13
84935,2021,Round 13,Optus Stadium,Daniel Rioli,Richmond,West Coast,Away,10443,60,West Coast,13
84901,2021,Round 13,Optus Stadium,Dominic Sheed,West Coast,Richmond,Home,10443,71,West Coast,13
84923,2021,Round 13,Optus Stadium,Dustin Martin,Richmond,West Coast,Away,10443,110,West Coast,13
84936,2021,Round 13,Optus Stadium,Dylan Grimes,Richmond,West Coast,Away,10443,91,West Coast,13
84905,2021,Round 13,Optus Stadium,Elliot Yeo,West Coast,Richmond,Home,10443,99,West Coast,13
84909,2021,Round 13,Optus Stadium,Harry Edwards,West Coast,Richmond,Home,10443,89,West Coast,13
84917,2021,Round 13,Optus Stadium,Jack Darling,West Coast,Richmond,Home,10443,27,West Coast,13


# 2022 Round 1 Melbourne v Western Bulldogs

In [213]:
url = 'https://www.footywire.com/afl/footy/supercoach_round?year=2022&round=1&p=&s=T'

download = pd.read_html(url)

In [214]:
r1_2022 = download[10]

In [215]:
r1_2022

,0,1,2,3,4,5,6
0,Rank,Player,Team,Current Salary,2022 R1 Salary,2022 R1 Score,*2022 R1 Value
1,1,Luke Parker,Swans,"$456,700","$591,600",167,28.2
2,2,Tom Green,Giants,"$606,400","$430,000",164,38.1
3,3,Christian Petracca,Demons,"$616,000","$606,200",163,26.9
4,4,Matt Rowell,Suns,"$517,800","$342,900",157,45.8
...,...,...,...,...,...,...,...
411,406,Tanner Bruhn,Giants,"$396,700","$263,300",0,0.0
412,406,Travis Colyer,Dockers,"$309,600","$323,600",0,0.0
413,406,Alex Sexton,Suns,"$139,800","$300,600",0,0.0
414,406,Isiah Winder,Eagles,"$176,200","$123,900",0,0.0


In [216]:
r1_2022 = clean_df(r1_2022, 'Melbourne', 'Western Bulldogs')

In [217]:
len(r1_2022)

46

In [218]:
idx_to_change = df_fw.loc[((df_fw['number'] == 1) & 
                          (df_fw['Season'] == 2022) &
                          ((df_fw['Team'] == "Melbourne") |
                          (df_fw['Team'] == "Western Bulldogs")))].sort_values('Player').index

# player missing
len(idx_to_change)

45

In [219]:
# Ugle-Hagan did not play
# Need to change Neal-Bullen to N-Bullen
set(r1_2022.player) - set(df_fw.loc[idx_to_change, 'Player'])

{'Alex Neal-Bullen', 'Jamarra Ugle-Hagan'}

In [220]:
r1_2022.loc[r1_2022['player'].str.contains('Neal'), 'player'] = 'Alex N-Bullen'
r1_2022.drop(409, inplace=True)


In [221]:
df_fw.loc[idx_to_change, 'SC'] = r1_2022['SC'].tolist()

In [222]:
df_fw.loc[idx_to_change]

,Season,Round,Venue,Player,Team,Opposition,Status,Match_id,SC,match_home_team,number
89306,2022,Round 1,MCG,Aaron Naughton,Western Bulldogs,Melbourne,Away,10544,108,Melbourne,1
89289,2022,Round 1,MCG,Adam Tomlinson,Melbourne,Western Bulldogs,Home,10544,39,Melbourne,1
89295,2022,Round 1,MCG,Adam Treloar,Western Bulldogs,Melbourne,Away,10544,79,Melbourne,1
89308,2022,Round 1,MCG,Alex Keath,Western Bulldogs,Melbourne,Away,10544,59,Melbourne,1
89275,2022,Round 1,MCG,Alex N-Bullen,Melbourne,Western Bulldogs,Home,10544,94,Melbourne,1
89272,2022,Round 1,MCG,Angus Brayshaw,Melbourne,Western Bulldogs,Home,10544,79,Melbourne,1
89297,2022,Round 1,MCG,Bailey Dale,Western Bulldogs,Melbourne,Away,10544,116,Melbourne,1
89293,2022,Round 1,MCG,Bailey Smith,Western Bulldogs,Melbourne,Away,10544,139,Melbourne,1
89303,2022,Round 1,MCG,Bailey Williams,Western Bulldogs,Melbourne,Away,10544,24,Melbourne,1
89286,2022,Round 1,MCG,Bayley Fritsch,Melbourne,Western Bulldogs,Home,10544,61,Melbourne,1


In [223]:
# all fixed
df_fw.isna().sum()

Season             0
Round              0
Venue              0
Player             0
Team               0
Opposition         0
Status             0
Match_id           0
SC                 0
match_home_team    0
number             0
dtype: int64

In [224]:
new_df = df_fw[['Season', 'number', 'match_home_team', 'Player', 'Team', 'Match_id', 'SC']]

In [225]:
new_df['SC'] = new_df['SC'].astype(float)

/var/folders/_d/tnkg6yhj7xj4rxgh61bmdwtr0000gn/T/ipykernel_97804/766329667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['SC'] = new_df['SC'].astype(float)


In [226]:
matches = pd.read_parquet('../../data/raw/cleaned_stats_12-22_fixed_bv.parquet').drop_duplicates('match_id')

In [227]:
matches = matches[['match_id', 'match_home_team', 'season', 'match_round']]

In [228]:
matches

,match_id,match_home_team,season,match_round
0,13960,Greater Western Sydney,2012,1
44,13961,Richmond,2012,1
88,13962,Hawthorn,2012,1
132,13963,Melbourne,2012,1
176,13964,Gold Coast,2012,1
...,...,...,...,...
8878,16342,Essendon,2022,23
8924,16343,Port Adelaide,2022,23
8970,16344,Hawthorn,2022,23
9016,16345,Carlton,2022,23


In [231]:
out_df = pd.merge(new_df, matches,
         left_on=['Season', 'number', 'match_home_team'],
         right_on=['season', 'match_round', 'match_home_team'],
         how='left').drop(columns=['Season', 'number', 'match_home_team', 'Match_id'])

In [235]:
# these 484 values correspond with na values found in previous notebook
out_df.isna().sum()

Player           0
Team             0
SC               0
match_id       484
season         484
match_round    484
dtype: int64

In [239]:
out_df.dropna(inplace=True)
len(out_df)

93743

In [238]:
out_df.to_parquet('../../data/raw/supercoach_12-22.parquet')